In [24]:
!pip install wget

In [25]:
import wget

url = "https://huggingface.co/datasets/Thefoodprocessor/allergy_type/resolve/main/data/train-00000-of-00001-eb2a4925fb8b17b7.parquet?download=true"

wget.download(url, "./train-00000-of-00001-eb2a4925fb8b17b7.parquet")

'./train-00000-of-00001-eb2a4925fb8b17b7 (1).parquet'

In [56]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
import re
from sklearn.preprocessing import MultiLabelBinarizer


df = pd.read_parquet("./train-00000-of-00001-eb2a4925fb8b17b7.parquet")

df.head()

,id,recipe,allergy_type
0,0,Roasted Carrots and Beets with Pecan Pesto\n1 ...,"[Nut, Beet, Fruit, Cheese]"
1,1,Escarole and Olive Salad with Raspberry Dressi...,"[Fruit, Vegetable, Nut, Sesame]"
2,2,Apple Tansie\n12 egg yolks\n6 egg whites\n1/2 ...,"[Egg, Fruit]"
3,3,Sauteed Foie Gras with Ginger Poached Pears an...,"[Milk, Nut, Soy, Wheat, Onion, Carrot, Leek, G..."
4,4,Lemon-Scented Broccolini\n1 1/2 pounds broccol...,[Lemon]


In [57]:
df.shape # (74465, 3)

(74465, 3)

In [58]:
x = df["recipe"]
y = df["allergy_type"]

print(x.shape, y.shape) # (74465,) (74465,)

(74465,) (74465,)


In [59]:
def clean_text(text):
    text = text.lower()  # Mettre en minuscule
    text = re.sub(r'\n', ' ', text)  # Supprimer les retours à la ligne
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)  # Supprimer caractères spéciaux
    return text

df["recipe"] = df["recipe"].astype(str).apply(clean_text)

print(df["recipe"].head())

0    roasted carrots and beets with pecan pesto 1 1...
1    escarole and olive salad with raspberry dressi...
2    apple tansie 12 egg yolks 6 egg whites 12 pint...
3    sauteed foie gras with ginger poached pears an...
4    lemonscented broccolini 1 12 pounds broccolini...
Name: recipe, dtype: object


In [60]:
def format_allergy_type(allergy_type): # Besoin de corrections pour la version finale
    # le code est actuellement structuré comme suit : [nut, beef, fruit, cheese]
    # on va le transformer en une liste de strings ["nut", "beef", "fruit", "cheese"]
    # on va aussi supprimer les espaces en trop
    allergy_type = allergy_type.replace("[", "").replace("]", "").replace(" ", "")
    allergy_type = allergy_type.split(",")
    return allergy_type

df["allergy_type"] = df["allergy_type"].apply(format_allergy_type)

df.head()

,id,recipe,allergy_type
0,0,roasted carrots and beets with pecan pesto 1 1...,"[Nut, Beet, Fruit, Cheese]"
1,1,escarole and olive salad with raspberry dressi...,"[Fruit, Vegetable, Nut, Sesame]"
2,2,apple tansie 12 egg yolks 6 egg whites 12 pint...,"[Egg, Fruit]"
3,3,sauteed foie gras with ginger poached pears an...,"[Milk, Nut, Soy, Wheat, Onion, Carrot, Leek, G..."
4,4,lemonscented broccolini 1 12 pounds broccolini...,[Lemon]


In [61]:
# S'assurer que les valeurs sont bien interprétées comme des listes
df["allergy_type"] = df["allergy_type"].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Extraire tous les allergènes uniques
unique_allergens = set(allergen for sublist in df["allergy_type"].dropna() for allergen in sublist)

# Afficher la liste des allergènes
print(sorted(unique_allergens))


['', 'Acid', 'Acidic', 'Acidity', 'Acornsquash', 'Adhesive', 'Adobo', 'Agave', 'Aioli', 'Alcohol', 'Alcohol(Bourbon)', 'Alcohol(bourbon)', 'AlcoholDisclaimer:*TheeggwhiteisusedasanemulsifierinthisrecipeIfyouareallergictoeggs', 'Ale', 'Alfalfa', 'AlfalfaSprouts', 'AlfonoOlives', 'AllSpice', 'Allergen', 'Allergens', 'Allergensnotspecified', 'Allergy', 'Alliacea', 'Allium', 'Allium(Garlic', 'Allium(garlic)', 'Allium(lavender)', 'AlliumNote:FrenchingisacookingtechniquewheremeatispartiallydebonedbycuttingawaythefattytissuebetweenthebonesInthiscase', 'Allspice', 'Almond', 'Almonds', 'Aloe', 'AmberBeer', 'Anatto', 'Anchochile', 'Anchovies', 'Anchovy', 'Animal', 'AnimalDerivatives', 'AnimalParts', 'AnimalProtein', 'Animalbyproduct', 'Animalproduct', 'Animalprotein', 'Animalprotein(Prawn)', 'Anise', 'Aniseed', 'Apple', 'AppleCiderVinegar', 'Applesauce', 'Apricot', 'ApricotPreserves', 'Arbolchile', 'Arrowroot', 'Artichoke', 'Artichokes', 'ArtificialColoring', 'Artificialcolors', 'Arugula', 'Asaf

In [51]:
print(df["allergy_type"].value_counts())


allergy_type
[Alcohol]                             555
[Dairy, Sugar]                        436
[Dairy, Fruit]                        427
[Dairy, Egg]                          408
[Milk, Egg]                           400
                                     ... 
[Nut, Dairy, Grain, Meat]               1
[Coconut, Eggs, Sugar]                  1
[Milk, Egg, Allium, Cheese, Grain]      1
[Cabbage, Butter]                       1
[Cow'smilk, Gluten, Garlic, Lemon]      1
Name: count, Length: 31228, dtype: int64


In [62]:
print(f"Original y shape: {df['allergy_type'].shape}")  # (74465,)

# S'assure que allergy_type est bien formaté comme une liste
df["allergy_type"] = df["allergy_type"].apply(lambda x: eval(x) if isinstance(x, str) else x)

# One-hot encode allergies
mlb = MultiLabelBinarizer() # applique les labels d'allergènes a plusieurs recettes
y = mlb.fit_transform(df["allergy_type"])

# Vérification de la nouvelle shape
print(f"New y shape: {y.shape}")  # (74465, number_of_allergens)


Original y shape: (74465,)
New y shape: (74465, 1691)


In [33]:
# import ast # Obsolete (peut-être réutiliser dans la version finale)
# df["allergy_type"] = df["allergy_type"].apply(ast.literal_eval)  # Convertie string list → vrai list


ValueError: malformed node or string: ['Nut', 'Beet', 'Fruit', 'Cheese']

In [63]:
from collections import Counter # permet de compter la fréquence d'éléments dans une liste

# Flatten la collone d'allergies pour compter le nombre de fois qu'elles sont listés dans toutes la df
allergy_counts = Counter(allergy for allergies in df["allergy_type"] for allergy in allergies)

# Création d'un autre dataframe pour une meilleur visualisaion
allergy_df = pd.DataFrame(allergy_counts.items(), columns=["Allergy", "Count"]).sort_values(by="Count", ascending=False)

print(allergy_df)
print(allergy_df.shape) # (1691, 2)


                                      Allergy  Count
7                                        Milk  19787
0                                         Nut  17233
21                                      Dairy  16635
6                                         Egg  14430
9                                       Wheat  13055
...                                       ...    ...
975   Soy(intheformofsoysauceinthesherrywine)      1
976                              JalapenoJack      1
977                           Soy(truffleoil)      1
980                           Tri-colorpepper      1
1690                     Wheat(inbreadcrumbs)      1

[1691 rows x 2 columns]
(1691, 2)


In [65]:
# print(df["allergy_type"].head(10))


0                           [Nut, Beet, Fruit, Cheese]
1                      [Fruit, Vegetable, Nut, Sesame]
2                                         [Egg, Fruit]
3    [Milk, Nut, Soy, Wheat, Onion, Carrot, Leek, G...
4                                              [Lemon]
5    [Flour, Butter, CreamCheese, Egg, Vanilla, Alm...
6                                  [Nut, Fruit, Grain]
7                                    [Nut, Dairy, Egg]
8                              [Fruit, Alcohol, Spice]
9                                   [Nut, Seed, Spice]
Name: allergy_type, dtype: object


In [71]:
# Vérification que tous les allérgènes sont dans une liste
df["allergy_type"] = df["allergy_type"].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Créer une matrice binaire des allérgènes
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df["allergy_type"])

# Création d'une dataframe de matrice binaire
y_df = pd.DataFrame(y, columns=mlb.classes_)

print(y_df.head())  # Vérification de la df
print(y_df.shape)   # (74465, 1691)


      Acid  Acidic  Acidity  Acornsquash  Adhesive  Adobo  Agave  Aioli  \
0  0     0       0        0            0         0      0      0      0   
1  0     0       0        0            0         0      0      0      0   
2  0     0       0        0            0         0      0      0      0   
3  0     0       0        0            0         0      0      0      0   
4  0     0       0        0            0         0      0      0      0   

   Alcohol  ...  whichisacommonallergen  \
0        0  ...                       0   
1        0  ...                       0   
2        0  ...                       0   
3        0  ...                       0   
4        0  ...                       0   

   whichisnottypicallyconsideredanallergenHowever  \
0                                               0   
1                                               0   
2                                               0   
3                                               0   
4                        

In [67]:
print(y_df.columns.tolist())  # Liste de tous les allérgènes possibles


['', 'Acid', 'Acidic', 'Acidity', 'Acornsquash', 'Adhesive', 'Adobo', 'Agave', 'Aioli', 'Alcohol', 'Alcohol(Bourbon)', 'Alcohol(bourbon)', 'AlcoholDisclaimer:*TheeggwhiteisusedasanemulsifierinthisrecipeIfyouareallergictoeggs', 'Ale', 'Alfalfa', 'AlfalfaSprouts', 'AlfonoOlives', 'AllSpice', 'Allergen', 'Allergens', 'Allergensnotspecified', 'Allergy', 'Alliacea', 'Allium', 'Allium(Garlic', 'Allium(garlic)', 'Allium(lavender)', 'AlliumNote:FrenchingisacookingtechniquewheremeatispartiallydebonedbycuttingawaythefattytissuebetweenthebonesInthiscase', 'Allspice', 'Almond', 'Almonds', 'Aloe', 'AmberBeer', 'Anatto', 'Anchochile', 'Anchovies', 'Anchovy', 'Animal', 'AnimalDerivatives', 'AnimalParts', 'AnimalProtein', 'Animalbyproduct', 'Animalproduct', 'Animalprotein', 'Animalprotein(Prawn)', 'Anise', 'Aniseed', 'Apple', 'AppleCiderVinegar', 'Applesauce', 'Apricot', 'ApricotPreserves', 'Arbolchile', 'Arrowroot', 'Artichoke', 'Artichokes', 'ArtificialColoring', 'Artificialcolors', 'Arugula', 'Asaf

In [70]:
unique_allergens = set(allergen for sublist in df["allergy_type"] for allergen in sublist)
print(sorted(unique_allergens))  # Vérification si il y'a des noms trop longs (à retravailler)


['', 'Acid', 'Acidic', 'Acidity', 'Acornsquash', 'Adhesive', 'Adobo', 'Agave', 'Aioli', 'Alcohol', 'Alcohol(Bourbon)', 'Alcohol(bourbon)', 'AlcoholDisclaimer:*TheeggwhiteisusedasanemulsifierinthisrecipeIfyouareallergictoeggs', 'Ale', 'Alfalfa', 'AlfalfaSprouts', 'AlfonoOlives', 'AllSpice', 'Allergen', 'Allergens', 'Allergensnotspecified', 'Allergy', 'Alliacea', 'Allium', 'Allium(Garlic', 'Allium(garlic)', 'Allium(lavender)', 'AlliumNote:FrenchingisacookingtechniquewheremeatispartiallydebonedbycuttingawaythefattytissuebetweenthebonesInthiscase', 'Allspice', 'Almond', 'Almonds', 'Aloe', 'AmberBeer', 'Anatto', 'Anchochile', 'Anchovies', 'Anchovy', 'Animal', 'AnimalDerivatives', 'AnimalParts', 'AnimalProtein', 'Animalbyproduct', 'Animalproduct', 'Animalprotein', 'Animalprotein(Prawn)', 'Anise', 'Aniseed', 'Apple', 'AppleCiderVinegar', 'Applesauce', 'Apricot', 'ApricotPreserves', 'Arbolchile', 'Arrowroot', 'Artichoke', 'Artichokes', 'ArtificialColoring', 'Artificialcolors', 'Arugula', 'Asaf

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer

# initialisation du TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words="english", max_features=5000)  # Limitation du nombres de features pour test
X = tfidf.fit_transform(df["recipe"])

print(X.shape)  # Should be (74465, nombre de features (5000))


(74465, 5000)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

# Split du dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_df, test_size=0.2, random_state=42)

# Initialisation du model
model = OneVsRestClassifier(LogisticRegression(max_iter=1000, solver="liblinear")) # à retravailler pour quelque chose de plus adapté a la solution

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluation du model

print(classification_report(y_test, y_pred))


In [107]:
import pickle

def save_model(filename, path):
    with open(f"{path}/{filename}.pkl", "wb") as file:
        pickle.dump(model, file)

save_model("model", "./")



In [39]:
# premiere solution (trop lente)

# # x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# from sklearn.model_selection import StratifiedKFold

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # utilisation de skf plutot que train test split, la méthode est plus adaptée quand certaines classes ont beaucoup plus d'exemples que d'autres

# # Permet de garantir que les données restent associés aux différents labels même une fois split et shuffle
# for train_index, test_index in skf.split(x, y.argmax(axis=1)):
#     x_train, x_test = x[train_index], x[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     break


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [40]:
# première solution (trop lente)

# print(x_train.shape, y_train.shape)
# print(x_test.shape, y_test.shape)

(59572,) (59572, 1691)
(14893,) (14893, 1691)


In [41]:
# première solution (trop lente)

# # Liste des labels manquants dans test et train
# labels_in_train = set(y_train.sum(axis=0).nonzero()[0])
# labels_in_test = set(y_test.sum(axis=0).nonzero()[0])

# # Liste des labels présent dans test mais pas dans train
# missing_labels = labels_in_test - labels_in_train
# print(f"Missing Labels in Training Set: {missing_labels}")

# # Retirer les labels manquants de test
# if missing_labels:
#     for label in missing_labels:
#         y_test[:, label] = 0  # Mis sur 0 par défault pour éviter les problémes de prédictions/évaluations


Missing Labels in Training Set: {514, 4, 1033, 1034, 1529, 526, 1038, 1551, 17, 1043, 20, 1557, 31, 545, 548, 1577, 554, 1073, 1587, 1588, 53, 55, 56, 569, 570, 1592, 1594, 573, 575, 1600, 579, 1603, 1605, 1609, 1610, 587, 1101, 1614, 79, 592, 594, 599, 88, 1112, 1625, 1627, 97, 609, 1634, 1126, 103, 106, 619, 1646, 1647, 1136, 1650, 1652, 632, 1660, 1661, 1150, 131, 646, 1674, 1675, 653, 146, 660, 662, 663, 1688, 1177, 1689, 670, 1184, 689, 1202, 1214, 1219, 708, 1221, 710, 199, 200, 1224, 1225, 206, 1237, 215, 727, 1242, 221, 743, 746, 747, 1259, 1261, 1262, 239, 753, 243, 1267, 245, 757, 1269, 249, 1274, 769, 1287, 1293, 273, 789, 791, 792, 283, 799, 298, 810, 1326, 1330, 1332, 1341, 318, 837, 838, 1350, 841, 1353, 1354, 1357, 1362, 854, 1366, 1370, 859, 1380, 1382, 1384, 361, 875, 882, 1395, 375, 1401, 378, 381, 902, 905, 1417, 396, 401, 1427, 404, 1433, 411, 1436, 926, 415, 416, 420, 933, 1447, 1448, 1451, 1454, 946, 439, 953, 955, 960, 451, 964, 454, 1478, 457, 1485, 1487, 467, 4

In [ ]:
# Première solution (trop lente)

# from sklearn.feature_extraction.text import TfidfVectorizer

# from sklearn.linear_model import SGDClassifier

# pipe = Pipeline([
#     ('vect', TfidfVectorizer()),
#     ('model', SGDClassifier(loss="log_loss", max_iter=1000, tol=1e-3, random_state=42, n_jobs=-1))
# ])


# pipe.fit(x_train, y_train)

In [105]:
# Fonction pour réaliser une prédiction avec le modèle
def predict_recipe(recipe_text):
    # Formattage de l'input
    recipe_text = clean_text(recipe_text)

    # Transformation du texte en vecteur numérique
    recipe_vec = tfidf.transform([recipe_text])

    # renvoi la prédiction du modèle
    prediction = model.predict(recipe_vec)

    # Conversion de la prédiction en labels lisibles
    predicted_allergies = mlb.inverse_transform(prediction)[0]

    # Formatage de la prédiction en chaine de caractères
    allergy_string = ", ".join(predicted_allergies)
    return f"Predicted allergy types: {allergy_string}"

# exemple 1
recipe_1 = "scrambled eggs" # oeufs brouillés
result_1 = predict_recipe(recipe_1)
print(result_1)

Predicted allergy types: Egg
